# Object Oriented Programming

This chapter is about object oriented design in Python. It covers the following things:

1. Object Oriented Design
2. Design Patterns
3. Class Definitions
4. Inheritance
5. Namespaces
6. Shallow and Deep copying

For each of these, I've documented what Python is doing

## 1. Object Oriented Design

![](images/intro_image.png)

## 2. Design Patterns

**Design Patterns** are solutions to solving "typical" software design problems:
* **Algorithm Design Patterns**: recursion, dynamic programming, divide and conquer, ...
* **Software Engineering Design Patterns**: iterator, factory method, template method pattern, ...

Some other ways to improve design:
* Using CRC cards ([My Notes](https://drive.google.com/open?id=1cFfzWrWslTACrP_rlJB0Lgfd-xYwPjMV)):
* Using UML Diagrams ([My Notes](https://drive.google.com/open?id=1lcylgtS136dCxLLpyKO1EnptSUD-Ub92)):
* Writing Pseudo-Code
* Maintaining good coding style ([PEP8](https://www.python.org/dev/peps/pep-0008/))
* Docstrings ([PEP257](http://www.python.org/dev/peps/pep-0257/))

## 3. Class Definitions

Each class has some number of instantiations, which each have their own attributes (state). 
In the class definition, we have the `self` identifier to identify which instance is being invoked.

Notice how you can assign attributes for the whole class, or qualify them using `self` so that they differ depending on the instance of the class.

In [1]:
class MyClass():
    class_val = 10                       # same for all members of the class
    def __init__(self, some_val):
        self.instance_val = some_val     # different for each class member
        
instance_1 = MyClass(10)
instance_2 = MyClass(42)

print(instance_1.instance_val, instance_1.class_val)
print(instance_2.instance_val, instance_2.class_val)

10 10
42 10


We can overload Python's **Special Methods** . 
See the `vector` class from the Python Intro notebook for an example of a class where we defined `+`, `len`, ...

The special methods are syntatic sugar: they call underlying methods, which are the ones you override in the class definition.
See below for some examples of this:

In [2]:
a = 2
b = 2
c = [1, 2, 3]
def d(x):
    return x+1
e = iter(c)
f = iter(c)

#  Operators
assert a + b == a.__add__(b)
assert a * b == a.__mul__(b)

#  Non-Operators
assert c[0] == c.__getitem__(0)
assert len(c) == c.__len__()
assert d(42) == d.__call__(42)
assert next(e) == f.__next__()  # need to initialize 2 iterators to test this

## 4. Inheritance

![](images/Exception_image.png)

Here is an example of a class `PredatoryCreditCard`, which inherits from superclass `CreditCard`

In [3]:
class CreditCard():
    __slots__ = '_balance', '_name', '_limit' 
    
    def __init__(self, name, limit=500):
        self._balance = 0
        self._name = name
        self._limit = limit
    def charge(self, price):
        """
        Charge given price to the card, assuming sufficient credit limit.
        Return True if charge was processed; False if charge was denied.
        """
        if price + self._balance > self._limit: # if charge would exceed limit,
            return False # cannot accept charge
        else:
            self._balance += price
            return True
        
    def make_payment(self, amount):
        """
        Process customer payment that reduces balance.
        """
        self._balance -= amount
    
    def get_balance(self):
        return('${}'.format(self._balance))


class PredatoryCreditCard(CreditCard):
    __slots__ = '_apr'
    
    OVERLIMIT_FEE = 5
    
    def __init__(self, name, limit=500, apr=0.15):
        super().__init__(name, limit)
        self._apr = apr
    
    def charge(self, price):
        success = super().charge(price)
        if not success:
            self._balance += PredatoryCreditCard.OVERLIMIT_FEE
        return success

## 5. Namespaces

![](images/Namespace_image.png)

In [4]:
cc_instance = CreditCard(name='john doe')
cc_instance.charge(4000)
print('result of transaction for cc_instance: {}'.format(cc_instance.get_balance()))

pcc_instance = PredatoryCreditCard(name='john doe', apr=0.08)
pcc_instance.charge(4000)
print('result of transaction for pcc_instance: {}'.format(pcc_instance.get_balance()))

result of transaction for cc_instance: $0
result of transaction for pcc_instance: $5


Notice how the `charge` method was changed in the subclass!

The namespaces of classes (as well as functions) can be accessed using `dir` and `var`.

In [5]:
print('in pcc_instance but not in cc_instance: {}'.format(set(dir(pcc_instance)).difference(set(dir(cc_instance)))))
print('in PredatoryCreditCard but not in CreditCard: {}'.format(set(dir(PredatoryCreditCard)).difference(set(dir(CreditCard)))))
print('in pcc_instance but not in PredatoryCreditCard: {}'.format(set(dir(pcc_instance)).difference(set(dir(PredatoryCreditCard)))))

in pcc_instance but not in cc_instance: {'OVERLIMIT_FEE', '_apr'}
in PredatoryCreditCard but not in CreditCard: {'OVERLIMIT_FEE', '_apr'}
in pcc_instance but not in PredatoryCreditCard: set()


Notice the differences in which methods appear in the class vs subclass namespaces.

Related but distinct of inheritance: you can have classes nested in other classes. This will be useful for Linked Lists, Trees, etc ...

In [6]:
## outer class
class Outer:

    ## inner class
    class Inner:
        pass

To retrieve a name in Pythons object-oriented framework (say, `foo.bar`), the interpreter:

   1. Searches the instance namespace. If the name is found, the value is used…
   2. Otherwise, searches the class namespace. If still not found…
   3. Searches upwards through the inheritance hierarchy… 
   4. If the name is not found, an `AttributeError` is raised.

Note that this means that methods and attributes of a superclass can be overridden by a subclass (which is what we did with the `charge` method above).


## 6. Shallow and Deep Copies

We create a `colors` class which contains attributes `red`, `green` and `blue`.
We assign identifier `warmtones` to an instantiation of the class.

In [7]:
import copy

class colors():
    def __init__(self, red, green, blue):
        self._red, self._green, self._blue = red, green, blue
    def __repr__(self):
        return '(' + str(self._red) + ',' + str(self._green) + ',' + str(self._blue) + ')'

warmtones = [colors(249, 124, 43), colors(169, 163, 52)]

We add the alias `palette`. Changing the underyling object of one changes the other, as expected.

In [8]:
palette = warmtones # alias

print('before: palette: {}'.format(palette))
print('before: warmtones: {}'.format(warmtones))

palette.append(colors(0, 0, 0))

print('after: palette: {}'.format(palette))
print('after: warmtones: {}'.format(warmtones)) # both change!

before: palette: [(249,124,43), (169,163,52)]
before: warmtones: [(249,124,43), (169,163,52)]
after: palette: [(249,124,43), (169,163,52), (0,0,0)]
after: warmtones: [(249,124,43), (169,163,52), (0,0,0)]


For shallow copies, the lists are seperate, so appending to one doesn't append to the other.
The objects in the list are aliases though, so changing one _does_ change the other!

In [9]:
warmtones = [colors(249, 124, 43), colors(169, 163, 52)]
        
palette = list(warmtones) # shallow copy

print('before: palette: {}'.format(palette))
print('before: warmtones: {}'.format(warmtones))

palette.append(colors(0, 0, 0))

print('after adding element: palette: {}'.format(palette))
print('after adding element: warmtones: {}'.format(warmtones)) # now only one changes!

palette[0]._red = 255 # change attribute of entry in one list

print('after mutating element: palette: {}'.format(palette))
print('after mutating element: warmtones: {}'.format(warmtones)) # both changes!


before: palette: [(249,124,43), (169,163,52)]
before: warmtones: [(249,124,43), (169,163,52)]
after adding element: palette: [(249,124,43), (169,163,52), (0,0,0)]
after adding element: warmtones: [(249,124,43), (169,163,52)]
after mutating element: palette: [(255,124,43), (169,163,52), (0,0,0)]
after mutating element: warmtones: [(255,124,43), (169,163,52)]


With deep copies this isn't the case. We can mutate the elements of one list without mutating the other.

In [10]:
warmtones = [colors(249, 124, 43), colors(169, 163, 52)]
palette = copy.deepcopy(warmtones) # deep copy

palette[0]._red = 255 # change attribute of entry in one list

print('palette: {}'.format(palette))
print('warmtones: {}'.format(warmtones)) # only one changes!

palette: [(255,124,43), (169,163,52)]
warmtones: [(249,124,43), (169,163,52)]


See this picture for a visualization of this: